Example of use of the Geocoding analysis.

We'll need to setup credentials that allow us to import a table and modify it; for convenience we'll used stored credentials, which you can save like this:

```python
from cartoframes.auth import Credentials
Credentials(username='...', api_key='...)).save()
```

In [1]:
from cartoframes.auth import set_default_credentials, Credentials
credentials = Credentials.from_file()
set_default_credentials(credentials)

Let's fetch a small dataset with addresses:

In [2]:
import pandas
dataframe = pandas.read_csv("https://storage.googleapis.com/test_carto/carto_places.csv")

In [3]:
dataframe

,place,address,country
0,CARTO NY,"114 W 26th St, New York, NY 10001",USA
1,CARTO Madrid,"Gran Via 46, 28013 Madrid",Spain
2,CARTO DC,"4075 Wilson Blvd, Arlington VA 22203",USA
3,CARTO Sevilla,"Calle Ebro 1, 41012 Sevilla",Spain


Now let's instantiate a Geocoder object

In [4]:
from cartoframes.data.services import Geocode
geocoder = Geocode()

Before doing any processing we can preview what needs to be geocoded and see how many credits will be spended:

In [7]:
_,preview = geocoder.geocode(dataframe, street='address', country='country', dry_run=True)

{'total_rows': 4,
 'required_quota': 4,
 'previously_geocoded': 0,
 'previously_failed': 0,
 'records_with_geometry': 0}

Note that *required_quota* is 4; now let's geocode the dataframe; we'll obtain a new dataframe with geometry column. It will keep information about the geocoding (a new column carto_geocode_hash) so that future geocodings don't need to spend credits for already-geocoded rows.

In [8]:
gc_dataframe, info = geocoder.geocode(dataframe, street='address', country='country', metadata='meta')
info

/Users/jgoizueta/.local/share/virtualenvs/cartoframes-voNRwPo2/src/carto/carto/sql.py:229: UserWarning: Batch SQL job created with job_id: 3b1db3a8-ef8e-4fc2-bb4e-5a40b454cc70
  warnings.warn('Batch SQL job created with job_id: {job_id}'.format(job_id=data['job_id']))


{'total_rows': 4,
 'required_quota': 4,
 'previously_geocoded': 0,
 'previously_failed': 0,
 'records_with_geometry': 0,
 'final_records_with_geometry': 4,
 'geocoded_increment': 4,
 'successfully_geocoded': 4,
 'failed_geocodings': 0}

Let's take a look at the results:

In [11]:
gc_dataframe

,place,address,country,carto_geocode_hash,meta,the_geom
cartodb_id,,,,,,
1,CARTO NY,"114 W 26th St, New York, NY 10001",USA,333cb1971579e7908f92d439a95d3ae3,"{""precision"": ""precise"", ""relevance"": 0.99, ""m...",0101000020E610000028B85851837F52C025404D2D5B5F...
2,CARTO Madrid,"Gran Via 46, 28013 Madrid",Spain,41acbd170c4dc9175e9cdda5a66c4351,"{""precision"": ""precise"", ""relevance"": 0.86, ""m...",0101000020E610000036B05582C5A10DC0A032FE7DC635...
3,CARTO DC,"4075 Wilson Blvd, Arlington VA 22203",USA,1e998c791d577302df42cefc40b03c74,"{""precision"": ""precise"", ""relevance"": 0.95, ""m...",0101000020E610000037A6272CF14653C031D3F6AFAC70...
4,CARTO Sevilla,"Calle Ebro 1, 41012 Sevilla",Spain,17c115e4b766018f635a9ea980da3783,"{""precision"": ""precise"", ""relevance"": 0.91, ""m...",0101000020E6100000912C6002B7EE17C0C45A7C0A80AD...


Now we realize that we should have stored the results into our CARTO account. No worries! We'll just repeate the geocoding operation adding a table_name parameter so that the results are stored into a table. Since the data is already geocoded (and we are preserving the hash column) it won't spend any credits.

In [ ]:
gc_dataset, info = geocoder.geocode(gc_dataframe, street='address', country='country', metadata='meta', table_name='carto_places')
info

In [14]:
from cartoframes.viz import Map, Layer
Map(Layer(gc_dataset))